In [ ]:
import numpy as np

from os.path import dirname, join

from bokeh.io import curdoc
from bokeh.layouts import column, row, layout
from bokeh.models import ColumnDataSource, Slider, TextInput, Toggle, Button, RadioButtonGroup, CustomJS
from bokeh.models import PreText, CheckboxGroup, Paragraph
from bokeh.plotting import figure
from bokeh.models.widgets import Tabs, Panel

from bokeh.models import Range1d

from bokeh.driving import count

import time
import serial

useSerial = False


# This portion of the code is for running the experiment

    
    
# Initialize the data lists, time is the same for all three plots
# use a Column Data Source, intially all lists are empty

source = ColumnDataSource(dict(timeList=[],
                               voltageList = [],
                               singleCountsA=[], 
                               singleCountsB=[], 
                               coincidenceCountsAB=[]
                              ))






# Set up user widgets

# Set up slider for user to choose counting time (100ms to 1000ms)
countingTimeSlider = Slider(start=0.1, end=20, value=1, step=0.1, title="Counting Time (s)")
countingTime = countingTimeSlider.value # initialize the variable to store the counting interval


# Set up START/STOP button, to start and stop counting
startStopButtonGroup = RadioButtonGroup(labels=["START", "STOP"], active=1)

# Set up a run experiment button. when clicked, experiment will run from start to finish
runExperimentButton = Button(label = 'Run Experiment', button_type = 'primary')

# Set up slider for user to set voltage range for piezoelectric
voltageRangeSlider = Slider(start=0, end=140, value=140, step=1, title="Voltage Range (V)")
voltageRange = voltageRangeSlider.value # initialize the variable to store the voltage range

# Set up slider for user to set voltage step for piezoelectric
voltageStepSlider = Slider(start=1, end=10, value=5, step=0.5, title="Voltage Step (V)")

# Set up the reset button, to clear the graph of all data
resetGraphButton = Button(label="Reset")

# Set up the save datda button, to export the data to a csv file
saveDataButton = Button(label="Save Data")

AHover = [
    ("Voltage: ", "@voltageList"),
    ("Counts: ", "@singleCountsA")
]

BHover = [
    ("Voltage: ", "@voltageList"),
    ("Counts: ", "@singleCountsB")
]

ABHover = [
    ("Voltage: ", "@voltageList"),
    ("Counts: ", "@coincidenceCountsAB")
]


# Create our 4 plots

coincidenceCountsvsVPlot = figure(plot_height=400, plot_width=800, title="Coincidence Counts vs Voltage",
              tools="crosshair,pan,reset,save,wheel_zoom", x_range = (0, voltageRange))

singleCountsAPlot = figure(plot_height=200, plot_width=600, title="Single Counts on Port A",
              tools="crosshair,pan,reset,save,wheel_zoom", x_range = (0, voltageRange), tooltips = AHover)

singleCountsBPlot = figure(plot_height=200, plot_width=600, title="Single Counts on Port B",
              tools="crosshair,pan,reset,save,wheel_zoom", x_range = (0, voltageRange), tooltips = BHover)

coincidenceCountsABPlot = figure(plot_height=200, plot_width=600, title="Coincidence Counts on Ports A and B",
              tools="crosshair,pan,reset,save,wheel_zoom", x_range = (0, voltageRange), tooltips = ABHover)

# Set y-axis labels for graphs
singleCountsAPlot.yaxis.axis_label = "Counts per " + str(countingTime) + " seconds"
singleCountsBPlot.yaxis.axis_label = "Counts per " + str(countingTime) + " seconds"
coincidenceCountsABPlot.yaxis.axis_label = "Counts per " + str(countingTime) + " seconds"

# Make the x_range of each plot the same as the voltage range value
#singleCountsAPlot.x_range = (0, voltageRange)
#singleCountsBPlot.x_range = (0, voltageRange)
#coincidenceCountsABPlot.x_range = (0, voltageRange)


# How many points should be displayed on screen at a time. Can turn this into a user input widget
numberPointsDisplayed = 20


# Make the plots display only the most recent "numberPointsDisplayed" points
# This way, the graph will scroll to show the newest points
#singleCountsAPlot.x_range.follow = "end"
#singleCountsAPlot.x_range.follow_interval = numberPointsDisplayed*countingTime
    
#singleCountsBPlot.x_range.follow = "end"
#singleCountsBPlot.x_range.follow_interval = numberPointsDisplayed*countingTime
    
#coincidenceCountsABPlot.x_range.follow = "end"
#coincidenceCountsABPlot.x_range.follow_interval = numberPointsDisplayed*countingTime





# Make each point of each plot a circle

coincidenceCountsvsVPlot.circle(x = 'voltageList', y = 'coincidenceCountsAB', 
                                source = source, line_width=3, line_alpha=0.6)

singleCountsAPlot.circle(x = 'voltageList', 
                         y = 'singleCountsA', 
                         source = source, 
                         line_width=3, 
                         line_alpha=0.6
                         )

singleCountsBPlot.circle(x = 'voltageList', 
                         y = 'singleCountsB', 
                         source = source, 
                         line_width=3, 
                         line_alpha=0.6
                         )

coincidenceCountsABPlot.circle(x = 'voltageList', 
                               y = 'coincidenceCountsAB', 
                               source = source, 
                               line_width=3, 
                               line_alpha=0.6
                               )

# Connect the points of each plot with a line
coincidenceCountsvsVPlot.line(x = 'voltageList', y = 'coincidenceCountsAB', 
                                source = source, line_width=3, line_alpha=0.6)

singleCountsAPlot.line(x = 'voltageList', 
                       y = 'singleCountsA', 
                       source = source, 
                       line_width=2, 
                       line_alpha=0.6
                       )

singleCountsBPlot.line(x = 'voltageList', 
                       y = 'singleCountsB', 
                       source = source, 
                       line_width=2, 
                       line_alpha=0.6
                      )

coincidenceCountsABPlot.line(x = 'voltageList', 
                             y = 'coincidenceCountsAB', 
                             source = source,
                             line_width=2, 
                             line_alpha=0.6
                            )


# set global variables to 0

# keep a running count of time elapsed, start at 0. This will help inform x-axis values.
runningTime = 0

numberDataPoints = 0

outputVoltage = 20

totalNumberDataPoints = 0

countingTime = 0
voltageRange = 0
voltageStep = 0

# Define a variable callback_id, which is attached to the periodic_callback function
callback_id = None


# Define a fuction to call when the current document (curdoc) has periodic callback on
# This function appends new data to our plots through the stream data command
def update_data():
    
    global numberDataPoints
    global totalNumberDataPoints
    
    
    # add data if total number of data points has not yet been reached
    if numberDataPoints < totalNumberDataPoints:
            
        # use the global variable
        global countingTime
        global voltageRange
        global voltageStep
        global runningTime
        global outputVoltage
        
        #countingTime = countingTimeSlider.value

        # add the counting time to our running time
        runningTime = runningTime + countingTime
        
        
        # the board ouput voltage number (0 - 255) is related to the desired output voltage in this way
        # round to the nearest whole number
        boardOutputVoltage = round((outputVoltage/49.3)/0.016)
        
        # send voltage command
        voltageCommand = "V" + str(boardOutputVoltage) + "\n"
        
        if useSerial == True:
            s.write(voltageCommand.encode())
            voltRead = s.readline()
        
        # the true output voltage is slightly different from desired output voltage due to rounding
        # it is based on the number command sent to board
        trueOutputVoltage = boardOutputVoltage*0.016*49.3

        if useSerial == True:
            # Request data from the CD48 counter
            s.write("c\n".encode())
            serialData = s.readline()
            counterData = [int(x) for x in serialData.decode('ascii').rstrip().split(' ')]

            # append one new data point to each list
            new_data = dict(
                timeList=[runningTime],
                voltageList = [trueOutputVoltage],
                singleCountsA=[counterData[0]],
                singleCountsB=[counterData[1]],
                coincidenceCountsAB=[counterData[2]]
                )
        else:
            # append one new data point to each list
            new_data = dict(
                timeList=[runningTime],
                voltageList = [trueOutputVoltage],
                singleCountsA=[0],
                singleCountsB=[0],
                coincidenceCountsAB=[0]
                )

        # Stream the new data to the plots.
        source.stream(new_data)

        # keep track of how many data points have been plotted
        numberDataPoints = numberDataPoints + 1
        
        # increase the desired output voltage by the voltage step
        outputVoltage = outputVoltage + voltageStep
    
    # else don't add new data
    else:
        
        # remove callback
        global callback_id
        curdoc().remove_periodic_callback(callback_id)
    
    
    





'''
# Define a function to be called when the START/STOP button changes value
def change_periodic_callback(new):
    # use global variable
    global callback_id
    
    # if the START button is on
    if startStopButtonGroup.active == 0:
        
        # Add a periodic_callback, the callback interval is stored in the variable countingTime
        #global countingTime
        countingTime = countingTimeSlider.value
        callback_id = curdoc().add_periodic_callback(update_data, countingTime)
    
    # else if the STOP button is on
    elif startStopButtonGroup.active == 1:
        
        # Remove the periodic callback
        curdoc().remove_periodic_callback(callback_id)
'''   

def run_experiment():
    
    global callback_id
    global numberDataPoints
    global outputVoltage
    global totalNumberDataPoints
    global runningTime
    
    global countingTime
    global voltageRange
    global voltageStep
    
    
    # retrieve slider values
    countingTime = countingTimeSlider.value
    voltageRange = voltageRangeSlider.value
    voltageStep = voltageStepSlider.value
    
    totalNumberDataPoints = int((voltageRange - 20)/voltageStep) + 1
    
    # voltage from the board goes from 0 to 4.08 with over command range 0 to 255
    # so you can output voltages at step 0.016V  (0.016 * 255 = 4.08,  0.016*250 = 4)
    
    #boardMaxOutputVoltage = voltageRange / 40
    
    numberDataPoints = 0
    
    outputVoltage = 20
    
    runningTime = 0
    
    if useSerial == True:
        # make a count to clear counters
        s.write("c\n".encode())
        serialData = s.readline()
    
    #curdoc().remove_periodic_callback(callback_id)
    callback_id = curdoc().add_periodic_callback(update_data, countingTime*1000)
    

# Define a function to be called when the reset button is pressed.   
def reset_plot():
    
    # set the running time back to 0
    global runningTime
    global numberDataPoints
    global outputVoltage
    global totalNumberDataPoints
    
    runningTime = 0
    
    numberDataPoints = 0

    outputVoltage = 0

    totalNumberDataPoints = 0
    
    # empty all data lists in source
    source.data = {k: [] for k in source.data}
    
    if useSerial == True:
        voltageCommand = "V" + str(0) + "\n"

        s.write(voltageCommand.encode())
        voltRead = s.readline()
    
'''
def change_callback_period(attr, old, new):
    # use global variable
    global callback_id
    
    # if the START button is on
    if startStopButtonGroup.active == 0:
        
        # Remove the periodic callback
        curdoc().remove_periodic_callback(callback_id)
        
        # Add a periodic_callback, the callback interval is stored in the variable countingTime
        global countingTime
        countingTime = countingTimeSlider.value
        callback_id = curdoc().add_periodic_callback(update_data, countingTime*1000)
'''        
        
    
# This part doesn't work yet
def change_voltage_range(attr, old, new):
    
    voltageRange = voltageRangeSlider.value
    
    singleCountsAPlot.x_range = Range1d(0, voltageRange)
    singleCountsBPlot.x_range = Range1d(0, voltageRange)
    coincidenceCountsABPlot.x_range = Range1d(0, voltageRange)
    
    
    
# Change y axis labels based on counting time value
def change_yaxis_labels(attr, old, new):
    
    countingTime = countingTimeSlider.value
    
    # Set y-axis labels for graphs
    singleCountsAPlot.yaxis.axis_label = "Counts per " + str(float(round(countingTime,1))) + " seconds"
    singleCountsBPlot.yaxis.axis_label = "Counts per " + str(float(round(countingTime,1))) + " seconds"
    coincidenceCountsABPlot.yaxis.axis_label = "Counts per " + str(float(round(countingTime,1))) + " seconds"
    
    
    
# When the START/STOP button is clicked, call the change_periodic_callback function
#startStopButtonGroup.on_click(change_periodic_callback)


# When the reset button is clicked, call the reset_plot function
resetGraphButton.on_click(reset_plot)

# run experiment when button is clicked
runExperimentButton.on_click(run_experiment)

# change y axis labels when counting time value changes
countingTimeSlider.on_change('value', change_yaxis_labels)

# change voltage range when voltage range value changes
voltageRangeSlider.on_change('value', change_voltage_range)


# When the save data button is clicked, export the data into a csv file
saveDataButton.js_on_click(CustomJS(args=dict(source=source),
                            code=open(join(dirname(__file__), "download.js")).read()))

# Set up layouts and add to document

# create a column of user input widgets
userInputWidgets = column(runExperimentButton,
                          voltageRangeSlider,
                          voltageStepSlider,
                          countingTimeSlider, 
                          resetGraphButton,
                          saveDataButton#,
                          #startStopButtonGroup
                         )




# create a column with our three plots
plotsColumn = column([coincidenceCountsABPlot,
                      singleCountsAPlot, 
                      singleCountsBPlot 
                     ])

# create a layout for the graphs interface, with the input widgets and the three plots
displayCountsLayout = layout([[row([userInputWidgets, plotsColumn])]] , sizing_mode='scale_both')



# create a tab for the graphs interface
displayCountsTab = Panel(child = displayCountsLayout, 
                         title="Graphs")

# combine all tabs into one layout
currentDocumentTabs = Tabs(tabs=[ displayCountsTab])


# Add tabs to current document
curdoc().add_root(currentDocumentTabs)

curdoc().title = "SinglePhotonInterterenceInterface"